Importación de librerías

In [1]:
import pandas as pd
import os

Carga de datos

In [2]:
data_dir = './data/'

proyectos_df = pd.read_csv(os.path.join(data_dir, 'proyectos.csv'))
contratos_df = pd.read_csv(os.path.join(data_dir, 'contratos.csv'))
pagos_df = pd.read_csv(os.path.join(data_dir, 'pagos.csv'))
ofertas_df = pd.read_csv(os.path.join(data_dir, 'ofertas.csv'))

C:\Users\crist\AppData\Local\Temp\ipykernel_13552\3524796897.py:5: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  pagos_df = pd.read_csv(os.path.join(data_dir, 'pagos.csv'))


Porcentaje de atraso de las obras públicas

In [3]:
contratos_df['ATRASO'] = (
    pd.to_datetime(contratos_df['FECHATERMINO']) - 
    pd.to_datetime(contratos_df['FECHATERMINOORIGINAL'])
).dt.days

obras_con_atraso = contratos_df[contratos_df['ATRASO'] > 0].shape[0]
total_obras = contratos_df.shape[0]
porcentaje_atraso = (obras_con_atraso / total_obras) * 100

print(f"Porcentaje de obras con atraso: {porcentaje_atraso:.2f}%")

C:\Users\crist\AppData\Local\Temp\ipykernel_13552\1990292342.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(contratos_df['FECHATERMINO']) -
C:\Users\crist\AppData\Local\Temp\ipykernel_13552\1990292342.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(contratos_df['FECHATERMINOORIGINAL'])


Porcentaje de obras con atraso: 37.07%


Tipo de obra que tarda más en realizarse

In [4]:
contratos_df['DURACION'] = (
    pd.to_datetime(contratos_df['FECHATERMINO']) - 
    pd.to_datetime(contratos_df['FECHAINICIO'])
).dt.days

tipo_obra_mas_lenta = contratos_df.groupby('SERVICIO')['DURACION'].mean().idxmax()

print(f"Tipo de obra que tarda más en realizarse: {tipo_obra_mas_lenta}")

C:\Users\crist\AppData\Local\Temp\ipykernel_13552\1180800161.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(contratos_df['FECHATERMINO']) -


Tipo de obra que tarda más en realizarse: VIALIDAD


C:\Users\crist\AppData\Local\Temp\ipykernel_13552\1180800161.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(contratos_df['FECHAINICIO'])


Tipo de obra más cara

In [5]:
tipo_obra_mas_cara = contratos_df.groupby('SERVICIO')['MONTOVIGENTE'].mean().idxmax()

print(f"Tipo de obra más cara: {tipo_obra_mas_cara}")

Tipo de obra más cara: VIALIDAD


Variación del presupuesto por año

In [ ]:
pagos_df['FECHAPAGO'] = pd.to_datetime(pagos_df['FECHAPAGO'])
pagos_df['ANIO'] = pagos_df['FECHAPAGO'].dt.year

presupuesto_anual = pagos_df.groupby('ANIO')['TOTALPAGADO'].sum()

#Convertimos esto a un dataframe

presupuesto = {}

presupuesto['Año'] = presupuesto_anual.index
presupuesto['Presupuesto'] = presupuesto_anual.values

presupuesto = pd.DataFrame(presupuesto)

#Datos de la inflación son sacados de https://www.datosmundial.com/america/chile/inflacion.php

inflacion_acumulada = {
    2023: 1.00,  # El año base
    2022: 1.0758,  # 1 + 7.58% de inflación
    2021: 1.1252,  # 1 + 12.52% acumulado hasta 2021
    2020: 1.1552,  # 1 + 15.52% acumulado hasta 2020
    2019: 1.1810,  # 1 + 18.10% acumulado hasta 2019
    2018: 1.1940,  # 1 + 19.40% acumulado hasta 2018
    2017: 1.2196,  # 1 + 21.96% acumulado hasta 2017
    2016: 1.1844,  # 1 + 18.44% acumulado hasta 2016
    2015: 1.1456,  # 1 + 14.56% acumulado hasta 2015
    2014: 1.1148,  # 1 + 11.48% acumulado hasta 2014
    2013: 1.1929,  # 1 + 19.29% acumulado hasta 2013
    2012: 1.1636,  # 1 + 16.36% acumulado hasta 2012
    2011: 1.1482,  # 1 + 14.82% acumulado hasta 2011
    2010: 1.0706   # 1 + 7.06% acumulado hasta 2010
}

#aquí básicamente convierto todos los presupuestos de años anteriores a lo que valdría ese presupuesto hoy 

presupuesto= presupuesto.drop(14)

def poner_puntos(numero): # le doy los puntitos bonitos
    return "{:,}".format(numero).replace(",", ".")

for x in range(len(presupuesto)):
    presupuesto.loc[x, 'Presupuesto'] = poner_puntos(round(presupuesto.loc[x, 'Presupuesto'] * inflacion_acumulada[int(presupuesto.loc[x, 'Año'])]))

presupuesto

ANIO
2010     248539801322
2011     580805918246
2012     796390115929
2013     938573697107
2014     982105439347
2015    1112733522537
2016    1047728165596
2017    1066038896559
2018    1022707634288
2019    1214177342638
2020    1144612156864
2021    1656678570279
2022    1718865918632
2023    1832500503599
2024    1014204031655
Name: TOTALPAGADO, dtype: int64


C:\Users\crist\AppData\Local\Temp\ipykernel_13552\3133368497.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pagos_df['FECHAPAGO'] = pd.to_datetime(pagos_df['FECHAPAGO'])


Empresa más contratada

In [7]:
empresa_mas_contratada = pagos_df['NOMBREADJUDICADO'].value_counts().idxmax()

print(f"Empresa más contratada: {empresa_mas_contratada}")

Empresa más contratada: CONST. DE PAVIMENTOS ASFALTICOS BITUMIX S.A.
